In [1]:
!pip install jupyter scikit-learn tensorflow tfx flask joblib tensorflow_model_analysis==0.46.0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tfx to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━

In [2]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

In [3]:
PIPELINE_NAME = "nandaaryaputra-pipeline"
SCHEMA_PIPELINE_NAME = "student-depression-prediction-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [4]:
DATA_ROOT = "/kaggle/input/student-depression-dataset/"

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion

## Load Dataset Using ExampleGen

In [6]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ])
)

# Load dataset from folder "data"
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output_config)

# run pipeline
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Data Validation

## Create summary statistic using StatisticsGen

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

## Create data schema using SchemaGen

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Academic Pressure',FLOAT,required,,-
'Age',FLOAT,required,,-
'CGPA',FLOAT,required,,-
'City',STRING,required,,'City'
'Degree',STRING,required,,'Degree'
'Depression',INT,required,,-
'Dietary Habits',STRING,required,,'Dietary Habits'
'Family History of Mental Illness',STRING,required,,'Family History of Mental Illness'
'Financial Stress',STRING,required,,'Financial Stress'


,Values
Domain,
'City',"'\'Less Delhi\'', '\'Less than 5 Kalyan\'', 'Agra', 'Ahmedabad', 'Bangalore', 'Bhavna', 'Bhopal', 'Chennai', 'City', 'Delhi', 'Faridabad', 'Gaurav', 'Ghaziabad', 'Harsh', 'Harsha', 'Hyderabad', 'Indore', 'Jaipur', 'Kalyan', 'Kanpur', 'Khaziabad', 'Kibara', 'Kolkata', 'Lucknow', 'Ludhiana', 'M.Com', 'M.Tech', 'ME', 'Meerut', 'Mihir', 'Mira', 'Mumbai', 'Nagpur', 'Nalini', 'Nalyan', 'Nandini', 'Nashik', 'Patna', 'Pune', 'Rajkot', 'Saanvi', 'Srinagar', 'Surat', 'Thane', 'Vadodara', 'Varanasi', 'Vasai-Virar', 'Visakhapatnam', '3.0', 'Rashi', 'Reyansh', 'Vaanya'"
'Degree',"'\'Class 12\'', 'B.Arch', 'B.Com', 'B.Ed', 'B.Pharm', 'B.Tech', 'BA', 'BBA', 'BCA', 'BE', 'BHM', 'BSc', 'LLB', 'LLM', 'M.Com', 'M.Ed', 'M.Pharm', 'M.Tech', 'MA', 'MBA', 'MBBS', 'MCA', 'MD', 'ME', 'MHM', 'MSc', 'Others', 'PhD'"
'Dietary Habits',"'Healthy', 'Moderate', 'Others', 'Unhealthy'"
'Family History of Mental Illness',"'No', 'Yes'"
'Financial Stress',"'1.0', '2.0', '3.0', '4.0', '5.0', '?'"
'Gender',"'Female', 'Male'"
'Have you ever had suicidal thoughts ?',"'No', 'Yes'"
'Profession',"'\'Civil Engineer\'', '\'Content Writer\'', '\'Digital Marketer\'', '\'Educational Consultant\'', '\'UX/UI Designer\'', 'Architect', 'Chef', 'Doctor', 'Entrepreneur', 'Lawyer', 'Manager', 'Pharmacist', 'Student', 'Teacher'"
'Sleep Duration',"'\'5-6 hours\'', '\'7-8 hours\'', '\'Less than 5 hours\'', '\'More than 8 hours\'', 'Others'"


## Identifying anomalies in dataset using ExampleValidator

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Data Preprocessing

## Create Preprocessing Function

In [13]:
TRANSFORM_MODULE_FILE = "dataset_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "Depression"
NUMERIC_FEATURES = ["Academic Pressure", "Age", "CGPA", "Study Satisfaction", "Work/Study Hours"]
CATEGORICAL_FEATURES = ["Dietary Habits", "Family History of Mental Illness", 
                        "Financial Stress", "Gender", "Have you ever had suicidal thoughts ?", 
                        "Sleep Duration"]

UNUSED_FEATURES = ["Job Satisfaction", "Work Pressure", "id", "City", "Profession", "Degree"]

def transformed_name(key):
    """Renames transformed features."""
    return key.replace("/", "_").replace(" ", "_").lower() + "_xf"

def preprocessing_fn(inputs):
    """
    Applies preprocessing to input features.

    Args:
        inputs: A dictionary mapping feature keys to raw tensors.

    Returns:
        outputs: A dictionary mapping feature keys to transformed tensors.
    """
    outputs = {}

    # Remove unused features
    filtered_inputs = {key: tensor for key, tensor in inputs.items() if key not in UNUSED_FEATURES}

    # Remove data where 'Financial Stress' is missing ("?")
    mask = tf.not_equal(filtered_inputs["Financial Stress"], "?")
    clean_inputs = {key: tf.boolean_mask(tensor, mask) for key, tensor in filtered_inputs.items()}

    # Standardize numerical features using Z-score normalization
    for feature in NUMERIC_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(clean_inputs[feature])

    # One-Hot Encoding for categorical features
    for feature in CATEGORICAL_FEATURES:
        vocab_filename = feature.replace(" ", "_").lower() + "_vocab"

        # Apply encoding using the created vocabulary
        indexed = tft.compute_and_apply_vocabulary(
            clean_inputs[feature],
            vocab_filename=vocab_filename,
            num_oov_buckets=1
        )

        vocab_size = tf.cast(tft.experimental.get_vocabulary_size_by_name(vocab_filename) + 1, tf.int32)

        # One-Hot Encoding
        one_hot_encoded = tf.one_hot(indexed, depth=vocab_size, on_value=1.0, off_value=0.0)

        outputs[transformed_name(feature)] = one_hot_encoded

    # Retain the target label without modification
    outputs[LABEL_KEY] = clean_inputs[LABEL_KEY]

    return outputs

Writing dataset_transform.py


## Transform data using Transform Component and Preprocessing Function

In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Model Training

## Tuner Hyperparameter

In [16]:
TUNER_MODULE_FILE = "model_tuner.py"

In [17]:
%%writefile {TUNER_MODULE_FILE}

import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Depression"
NUMERIC_FEATURES = ["Academic Pressure", "Age", "CGPA", "Study Satisfaction", "Work/Study Hours"]
CATEGORICAL_FEATURES = ["Dietary Habits", "Family History of Mental Illness", 
                        "Financial Stress", "Gender", "Have you ever had suicidal thoughts ?", "Sleep Duration"]
NUM_EPOCHS = 10

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

def transformed_name(key):
    return key.replace("/", "_").replace(" ", "_").lower() + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)
    return dataset

def model_builder(hp, tf_transform_output):
    inputs = {}
    numeric_inputs = [tf.keras.Input(shape=(1,), name=transformed_name(f), dtype=tf.float32) for f in NUMERIC_FEATURES]
    for f in NUMERIC_FEATURES:
        inputs[transformed_name(f)] = numeric_inputs[NUMERIC_FEATURES.index(f)]
    
    concat_numeric = layers.concatenate(numeric_inputs)
    x = layers.Dense(hp.Int("dense_units_1", min_value=32, max_value=128, step=32), activation='relu')(concat_numeric)
    
    categorical_inputs = []
    for feature in CATEGORICAL_FEATURES:
        transformed_feature_name = transformed_name(feature)
        vocab_size = tf_transform_output.vocabulary_size_by_name(feature.replace(" ", "_").lower() + "_vocab") + 1
        
        categorical_input = tf.keras.Input(shape=(vocab_size,), name=transformed_feature_name, dtype=tf.float32)
        categorical_inputs.append(categorical_input)
        inputs[transformed_feature_name] = categorical_input
    
    concat_categorical = layers.concatenate(categorical_inputs)
    combined = layers.concatenate([x, concat_categorical])
    x = layers.Dense(hp.Int("dense_units_2", min_value=32, max_value=128, step=32), activation='relu')(combined)
    x = layers.Dropout(hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return model

def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files, tf_transform_output, NUM_EPOCHS)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, NUM_EPOCHS)
    
    tuner = kt.BayesianOptimization(
        hypermodel=lambda hp: model_builder(hp, tf_transform_output),
        objective=kt.Objective('binary_accuracy', direction='max'),
        max_trials=30,
        directory=fn_args.working_dir,
        project_name="bayesian_tuning",
    )
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": eval_set,
            "epochs": NUM_EPOCHS,
        },
    )

Writing model_tuner.py


In [18]:
from tfx.proto import trainer_pb2

tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)
interactive_context.run(tuner)

Trial 30 Complete [00h 02m 28s]
binary_accuracy: 0.8817078471183777

Best binary_accuracy So Far: 0.8839736580848694
Total elapsed time: 01h 14m 37s
Results summary
Results in pipelines/nandaaryaputra-pipeline/.temp/6/bayesian_tuning
Showing 10 best trials
Objective(name="binary_accuracy", direction="max")

Trial 17 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8839736580848694

Trial 27 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8835303783416748

Trial 21 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.881900429725647

Trial 29 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8817078471183777

Trial 19 summary
Hyperparameters:
dense_units_1: 128
dense_units_2: 128
dropout_rate: 0.1
learning_rate: 0.001
Score: 0.8814436793327332

Tr

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Trainer

In [19]:
TRAINER_MODULE_FILE = "model_trainer.py"

In [20]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Depression"
NUMERIC_FEATURES = ["Academic Pressure", "Age", "CGPA", "Study Satisfaction", "Work/Study Hours"]
CATEGORICAL_FEATURES = ["Dietary Habits", "Family History of Mental Illness", 
                        "Financial Stress", "Gender", "Have you ever had suicidal thoughts ?", "Sleep Duration"]

def transformed_name(key):
    """Convert feature name to match transformed feature in preprocessing."""
    return key.replace("/", "_").replace(" ", "_").lower() + "_xf"

def gzip_reader_fn(filenames):
    """Read compressed TFRecord dataset."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    """Load and parse transformed dataset."""
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)
    return dataset

def model_builder(tf_transform_output):
    """Build the deep learning model using transformed features."""
    inputs = {}
    
    # Numeric Features (already normalized)
    numeric_inputs = [tf.keras.Input(shape=(1,), name=transformed_name(f), dtype=tf.float32) for f in NUMERIC_FEATURES]
    for f in NUMERIC_FEATURES:
        inputs[transformed_name(f)] = numeric_inputs[NUMERIC_FEATURES.index(f)]
    
    # Concatenate numeric inputs
    concat_numeric = layers.concatenate(numeric_inputs)
    x = layers.Dense(64, activation='relu')(concat_numeric)
    x = layers.Dense(32, activation='relu')(x)

    # Categorical Features (One-Hot Encoded)
    categorical_inputs = []
    for feature in CATEGORICAL_FEATURES:
        transformed_feature_name = transformed_name(feature)
        vocab_size = tf_transform_output.vocabulary_size_by_name(feature.replace(" ", "_").lower() + "_vocab") + 1
        
        categorical_input = tf.keras.Input(shape=(vocab_size,), name=transformed_feature_name, dtype=tf.float32)
        categorical_inputs.append(categorical_input)
        inputs[transformed_feature_name] = categorical_input

    # Concatenate categorical inputs
    concat_categorical = layers.concatenate(categorical_inputs)
    
    # Combine Numeric & Categorical Features
    combined = layers.concatenate([x, concat_categorical])
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    model.summary()
    return model

def _get_serve_tf_example_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        
        return model(transformed_features)
    
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    """Train and save the model."""
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(fn_args.serving_model_dir, "best_model.keras"),
        monitor='val_binary_accuracy',
        mode='max',
        save_best_only=True
    )
    
    # Load TFT transform graph
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Load training and validation datasets
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)
    
    # Build and train the model
    model = model_builder(tf_transform_output)
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        epochs=10)

    signatures = {
    "serving_default": _get_serve_tf_example_fn(
        model, tf_transform_output
    ).get_concrete_function(
        tf.TensorSpec(
            shape=[None], 
            dtype=tf.string, 
            name="examples"
        )
    )}
    
    # Save final model
    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)

Writing model_trainer.py


In [21]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs["best_hyperparameters"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),
)

interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 academic_pressure_xf (Inpu  [(None, 1)]                  0         []                            
 tLayer)                                                                                          
                                                                                                  
 age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 cgpa_xf (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 study_satisfaction_xf (Inp  [(None, 1)]                  0         []                      

ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Model Analysis and Validation

## Get model using Resolver Component

In [22]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Evaluate model using Evaluator Component

In [23]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Depression')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [24]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [25]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'example_cou…

## Export Model Using Pusher Component

In [26]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                SERVING_MODEL_DIR, "student-depression-prediction-model"
            ),
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Create Requirement.txt

In [27]:
!pip freeze >> requirements.txt